# Decision Optimisation for Continuous Outcomes

- skip_exec: true


In [ ]:
import math
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from quantile_forest import RandomForestQuantileRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

pd.set_option("display.max_columns", None)

PROJECT_ROOT = Path.cwd().parent.parent

plt.rcParams["figure.facecolor"] = (1, 1, 1, 0)  # RGBA tuple with alpha=0
plt.rcParams["axes.facecolor"] = (1, 1, 1, 0)  # RGBA tuple with alpha=0


The data that we will use comes from the [Grupo Bimbo Inventory Demand](https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand) Kaggle competition.

The goal is to predict the demand of a product for a given week, at a particular store (column **Demanda_uni_equil**). The dataset consists of 9 weeks of sales transactions in Mexico. Each transaction consists of sales and returns. Returns are the products that are unsold and expired. The demand for a product in a certain week is defined as the sales this week subtracted by the return next week.


In [ ]:
data = pd.read_csv(f"{PROJECT_ROOT}/data/grupo-bimbo-inventory-demand/train.csv", nrows=200000, low_memory=False)
clientes = pd.read_csv(f"{PROJECT_ROOT}/data/grupo-bimbo-inventory-demand/cliente_tabla.csv", low_memory=False)
productos = pd.read_csv(f"{PROJECT_ROOT}/data/grupo-bimbo-inventory-demand/producto_tabla.csv", low_memory=False)
town_state = pd.read_csv(f"{PROJECT_ROOT}/data/grupo-bimbo-inventory-demand/town_state.csv", low_memory=False)

data = pd.merge(data, clientes, on="Cliente_ID", how="left")
data = pd.merge(data, productos, on="Producto_ID", how="left")
data = pd.merge(data, town_state, on="Agencia_ID", how="left")


In [ ]:
data


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,NombreCliente,NombreProducto,Town,State
0,3,1110,7,3301,15766,1212,3,25.14,0,0.0,3,PUESTO DE PERIODICOS LAZARO,Roles Canela 2p 120g BIM 1212,2008 AG. LAGO FILT,"MÉXICO, D.F."
1,3,1110,7,3301,15766,1216,4,33.52,0,0.0,4,PUESTO DE PERIODICOS LAZARO,Roles Glass 2p 135g BIM 1216,2008 AG. LAGO FILT,"MÉXICO, D.F."
2,3,1110,7,3301,15766,1238,4,39.32,0,0.0,4,PUESTO DE PERIODICOS LAZARO,Panquecito Gota Choc 2p 140g BIM 1238,2008 AG. LAGO FILT,"MÉXICO, D.F."
3,3,1110,7,3301,15766,1240,4,33.52,0,0.0,4,PUESTO DE PERIODICOS LAZARO,Mantecadas Vainilla 4p 125g BIM 1240,2008 AG. LAGO FILT,"MÉXICO, D.F."
4,3,1110,7,3301,15766,1242,3,22.92,0,0.0,3,PUESTO DE PERIODICOS LAZARO,Donitas Espolvoreadas 6p 105g BIM 1242,2008 AG. LAGO FILT,"MÉXICO, D.F."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200735,3,1116,1,1466,2309869,1238,8,78.64,0,0.0,8,UNION DEL VALLE 2,Panquecito Gota Choc 2p 140g BIM 1238,2011 AG. SAN ANTONIO,"MÉXICO, D.F."
200736,3,1116,1,1466,2309869,1240,8,67.04,0,0.0,8,UNION DEL VALLE 2,Mantecadas Vainilla 4p 125g BIM 1240,2011 AG. SAN ANTONIO,"MÉXICO, D.F."
200737,3,1116,1,1466,2309869,1242,6,45.84,0,0.0,6,UNION DEL VALLE 2,Donitas Espolvoreadas 6p 105g BIM 1242,2011 AG. SAN ANTONIO,"MÉXICO, D.F."
200738,3,1116,1,1466,2309869,1250,27,206.28,0,0.0,27,UNION DEL VALLE 2,Donas Azucar 4p 105g BIM 1250,2011 AG. SAN ANTONIO,"MÉXICO, D.F."


In [ ]:
categorical_cols = ["Agencia_ID", "Canal_ID", "Ruta_SAK", "Cliente_ID", "Producto_ID"]

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.transform(data[col])
    label_encoders[col] = le

In [ ]:
num_unique_vals = {col: data[col].nunique() for col in categorical_cols}
embedding_sizes = {col: min(50, num_unique_vals[col] // 2) for col in categorical_cols}


In [ ]:
num_unique_vals

{'Agencia_ID': 6,
 'Canal_ID': 6,
 'Ruta_SAK': 343,
 'Cliente_ID': 10472,
 'Producto_ID': 478}

In [ ]:
embedding_sizes

{'Agencia_ID': 3,
 'Canal_ID': 3,
 'Ruta_SAK': 50,
 'Cliente_ID': 50,
 'Producto_ID': 50}

In [ ]:
X = data[categorical_cols].values
y = data["Demanda_uni_equil"].values

In [ ]:
X


array([[   0,    3,  293,    3,   43],
       [   0,    3,  293,    3,   44],
       [   0,    3,  293,    3,   48],
       ...,
       [   5,    0,  235, 7722,   50],
       [   5,    0,  235, 7722,   51],
       [   5,    0,  235, 7722,   53]])

In [ ]:
y


array([ 3,  4,  4, ...,  6, 27, 13])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
class BimboDataset(Dataset):
    def __init__(self, X, y):
        self.X = [torch.tensor(X[:, i], dtype=torch.long) for i in range(X.shape[1])]
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return [x[idx] for x in self.X], self.y[idx]

In [ ]:
train_dataset = BimboDataset(X_train, y_train)
val_dataset = BimboDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, embedding_sizes, hidden_size=128):
        super(SimpleModel, self).__init__()
        self.embeddings = nn.ModuleList(
            [nn.Embedding(num_unique_vals[col], embedding_sizes[col]) for col in categorical_cols]
        )
        self.fc1 = nn.Linear(sum(embedding_sizes.values()), hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = [embedding(x_i) for x_i, embedding in zip(x, self.embeddings)]
        x = torch.cat(x, dim=-1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x).squeeze(-1)
        return x


In [ ]:
def train_model(loss_fn, num_epochs=5):
    model = SimpleModel(embedding_sizes)
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_targets = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs).squeeze()
                loss = loss_fn(outputs, targets)
                val_loss += loss.item()
                val_preds.extend(outputs.tolist())
                val_targets.extend(targets.tolist())

        val_loss /= len(val_loader)
        r2 = r2_score(val_targets, val_preds)
        val_preds = np.clip(val_preds, 0, None)
        rmsle = np.sqrt(mean_squared_log_error(val_targets, val_preds))
        print(
            {
                "epoch": epoch,
                "train_loss": round(train_loss, 5),
                "val_loss": round(val_loss, 5),
                "r_squared": round(r2, 5),
                "rmsle": round(rmsle, 5),
            }
        )
    return model, np.array(val_preds), np.array(val_targets)

In [ ]:
business_metrics = pd.DataFrame(
    columns=[
        "Model Name",
        "Understocked Fraction",
        "Understocked Amount",
        "Overstocked Fraction",
        "Overstocked Amount",
        "Utility",
        "MAE",
        "MSE",
        "R2",
        "RMSLE",
    ]
)

In [ ]:
def log_business_metrics(model_name: str, stocking_decisions, actual_demand):
    frac_understocks = (stocking_decisions < actual_demand).mean()
    total_understocked_amt = (actual_demand - stocking_decisions).clip(0).sum()
    frac_overstocks = (stocking_decisions > actual_demand).mean()
    total_overstocked_amt = (stocking_decisions - actual_demand).clip(0).sum()
    utility = -3 * total_understocked_amt - total_overstocked_amt
    mae = mean_absolute_error(actual_demand, stocking_decisions)
    mse = mean_squared_error(actual_demand, stocking_decisions)
    r2 = r2_score(actual_demand, stocking_decisions)
    rmsle = np.sqrt(mean_squared_log_error(actual_demand, stocking_decisions))

    df = pd.DataFrame(
        data={
            "Model Name": model_name,
            "Understocked Fraction": frac_understocks,
            "Understocked Amount": total_understocked_amt,
            "Overstocked Fraction": frac_overstocks,
            "Overstocked Amount": total_overstocked_amt,
            "Utility": utility,
            "MAE": mae,
            "MSE": mse,
            "R2": r2,
            "RMSLE": rmsle,
        },
        index=[0],
    )

    return df


In [ ]:
loss = nn.MSELoss()
mse_model, mse_val_preds, mse_val_targets = train_model(loss, num_epochs=5)


{'epoch': 0, 'train_loss': 381.65047, 'val_loss': 207.07174, 'r_squared': 0.66431, 'rmsle': 0.65405}
{'epoch': 1, 'train_loss': 252.30558, 'val_loss': 210.79833, 'r_squared': 0.6582, 'rmsle': 0.63175}
{'epoch': 2, 'train_loss': 198.14795, 'val_loss': 228.69794, 'r_squared': 0.62927, 'rmsle': 0.5887}
{'epoch': 3, 'train_loss': 172.12081, 'val_loss': 189.04469, 'r_squared': 0.69348, 'rmsle': 0.59622}
{'epoch': 4, 'train_loss': 140.2638, 'val_loss': 169.67012, 'r_squared': 0.72494, 'rmsle': 0.58938}


In [ ]:
mse_val_stock = np.ceil(mse_val_preds)
bm1 = log_business_metrics("MSE model", mse_val_stock, mse_val_targets)
pd.concat([business_metrics, bm1], axis=0, ignore_index=True)

,Model Name,Understocked Fraction,Understocked Amount,Overstocked Fraction,Overstocked Amount,Utility,MAE,MSE,R2,RMSLE
0,MSE model,0.263425,78859.0,0.644864,101062.0,-337639.0,4.481444,170.217072,0.724259,0.637018


With the stocking rules above, we understock 27% of the time. This seems bad so let's try a different approach to making the stocking decision.

Below we take the existing predictions and multiply them by 1.5 and round them up. In other words we are going to stock 50% above the model's predictions and see what happens.


In [ ]:
alternative_stocking_rule = np.ceil(1.5 * mse_val_preds)
bm2 = log_business_metrics("MSE model * 1.5", alternative_stocking_rule, mse_val_targets)
pd.concat([business_metrics, bm1, bm2], axis=0, ignore_index=True)


,Model Name,Understocked Fraction,Understocked Amount,Overstocked Fraction,Overstocked Amount,Utility,MAE,MSE,R2,RMSLE
0,MSE model,0.263425,78859.0,0.644864,101062.0,-337639.0,4.481444,170.217072,0.724259,0.637018
1,MSE model * 1.5,0.128350,36920.0,0.817077,218203.0,-328963.0,6.354563,297.114402,0.518694,0.796495


We now understock only 8% of the time. We paid for this by increasing the percentage of weeks we are overstocked from 59% to 80%.

Looking at how the size of the overstocks we can se that we went from 89731 to 208034. In other words we've more than doubled how much unnecessary stuff we are buying. This suggests we would best off focusing improvements to our model that bring down this number.


### Using an alternaive loss function (L1 loss)


Currently our model is attempting to minimise the mean squared error. This is a sensible choice for many problems but it is not the only choice. For example, we could instead try to minimise the mean absolute error. This is known as the L1 loss.


In [ ]:
loss = nn.L1Loss()
mae_model, mae_val_preds, mae_val_targets = train_model(loss, num_epochs=5)

{'epoch': 0, 'train_loss': 4.91512, 'val_loss': 4.18438, 'r_squared': 0.52323, 'rmsle': 0.55182}
{'epoch': 1, 'train_loss': 4.00184, 'val_loss': 4.03253, 'r_squared': 0.62863, 'rmsle': 0.53773}
{'epoch': 2, 'train_loss': 3.71826, 'val_loss': 3.81097, 'r_squared': 0.64277, 'rmsle': 0.5199}
{'epoch': 3, 'train_loss': 3.54705, 'val_loss': 3.76971, 'r_squared': 0.68272, 'rmsle': 0.51888}
{'epoch': 4, 'train_loss': 3.39178, 'val_loss': 3.75299, 'r_squared': 0.68885, 'rmsle': 0.51394}


We can't really compare the the training and validation loss of in this model run to the previous one because they are using different loss functions, one of which squares its values and one of which doesn't. However, can look at what each decisions each model would have led us to and compare the resulting business metrics.


In [ ]:
mae_val_stock = np.ceil(mae_val_preds)
bm3 = log_business_metrics("MAE model", mae_val_stock, mae_val_targets)
pd.concat([business_metrics, bm1, bm2, bm3], axis=0, ignore_index=True)

,Model Name,Understocked Fraction,Understocked Amount,Overstocked Fraction,Overstocked Amount,Utility,MAE,MSE,R2,RMSLE
0,MSE model,0.263425,78859.0,0.644864,101062.0,-337639.0,4.481444,170.217072,0.724259,0.637018
1,MSE model * 1.5,0.128350,36920.0,0.817077,218203.0,-328963.0,6.354563,297.114402,0.518694,0.796495
2,MAE model,0.353069,90968.0,0.480746,60409.0,-333313.0,3.770474,191.123319,0.690392,0.521193


Let's also get the business metrics for when we stock 50% above the model's predictions.


In [ ]:
above_mae_stocking_rule = np.ceil(1.5 * mae_val_preds)
bm4 = log_business_metrics("MAE model * 1.5", above_mae_stocking_rule, mse_val_targets)
pd.concat([business_metrics, bm1, bm2, bm3, bm4], axis=0, ignore_index=True)

,Model Name,Understocked Fraction,Understocked Amount,Overstocked Fraction,Overstocked Amount,Utility,MAE,MSE,R2,RMSLE
0,MSE model,0.263425,78859.0,0.644864,101062.0,-337639.0,4.481444,170.217072,0.724259,0.637018
1,MSE model * 1.5,0.128350,36920.0,0.817077,218203.0,-328963.0,6.354563,297.114402,0.518694,0.796495
2,MAE model,0.353069,90968.0,0.480746,60409.0,-333313.0,3.770474,191.123319,0.690392,0.521193
3,MAE model * 1.5,0.170818,42610.0,0.731892,156209.0,-284039.0,4.952152,204.244570,0.669137,0.640890


We can see here that we both understock and overstock less often with a model trained with L1 loss. So swapping this in for MSE loss seems like a strict improvement.


### Going further with a custom loss function


We have metric, `Utility`, which we are defining as $-3(\text{Understocked Amount} - \text{Overstocked Amount})$. In the real world you would spend a lot of time deciding how to define a metric like this. Assuming that you have done that, you may decide to train your model attempt to optimise that.


In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, outputs, actual):
        diff = outputs - actual
        loss = torch.where(outputs > actual, diff, -3 * diff)
        return loss.mean()


In [ ]:
custom_model, custom_val_preds, custom_val_targets = train_model(CustomLoss(), num_epochs=5)


{'epoch': 0, 'train_loss': 9.82188, 'val_loss': 7.97964, 'r_squared': 0.50757, 'rmsle': 0.6809}
{'epoch': 1, 'train_loss': 7.55593, 'val_loss': 7.39512, 'r_squared': 0.62763, 'rmsle': 0.61739}
{'epoch': 2, 'train_loss': 6.89639, 'val_loss': 7.41605, 'r_squared': 0.43547, 'rmsle': 0.59855}
{'epoch': 3, 'train_loss': 6.49166, 'val_loss': 7.20923, 'r_squared': 0.5159, 'rmsle': 0.60815}
{'epoch': 4, 'train_loss': 6.07987, 'val_loss': 7.1619, 'r_squared': 0.70741, 'rmsle': 0.59515}


In [ ]:
custom_val_stock = np.ceil(custom_val_preds)
bm5 = log_business_metrics("Custom loss", custom_val_stock, custom_val_targets)
pd.concat([business_metrics, bm1, bm2, bm3, bm4, bm5], axis=0, ignore_index=True)

,Model Name,Understocked Fraction,Understocked Amount,Overstocked Fraction,Overstocked Amount,Utility,MAE,MSE,R2,RMSLE
0,MSE model,0.263425,78859.0,0.644864,101062.0,-337639.0,4.481444,170.217072,0.724259,0.637018
1,MSE model * 1.5,0.128350,36920.0,0.817077,218203.0,-328963.0,6.354563,297.114402,0.518694,0.796495
2,MAE model,0.353069,90968.0,0.480746,60409.0,-333313.0,3.770474,191.123319,0.690392,0.521193
3,MAE model * 1.5,0.170818,42610.0,0.731892,156209.0,-284039.0,4.952152,204.244570,0.669137,0.640890
4,Custom loss,0.184218,47662.0,0.724146,144332.0,-287318.0,4.782156,182.878250,0.703749,0.640359


We end up with a better Utility score when optimising for it directly. The challenge then is knowing whether that was the correct thing to optimise for in the first place.


## Predicting Full Distributions


A typical random forest is composed of decision trees. Each decision will take the training data and split it many times until it reaches a leaf. When you want to make predictions at inference time you will take a row of data, run it through a tree, and get a prediction which will be the mean or median of the data in whichever leaf the row ends up in. The random forest's final prediction is then the mean of each decision tree's predictions.

In a quantile random forest, if you want to know the median of the data in a leaf you will take the median of the data in that leaf. If you want to know the 90th percentile of the data in a leaf you will take the 90th percentile of the data in that leaf.


In [ ]:
qrf = RandomForestQuantileRegressor(n_estimators=100, min_samples_leaf=50, random_state=0)
qrf.fit(X_train, y_train)


RandomForestQuantileRegressor(min_samples_leaf=50, random_state=0)

In [ ]:
quantiles = [i / 100 for i in range(5, 100, 5)]
sample_preds = qrf.predict(X_val, quantiles=quantiles)


In [ ]:
sample_preds.shape

(40148, 19)

In [ ]:
sample_preds


array([[ 1.  ,  1.  ,  1.  , ...,  7.  ,  8.  , 10.  ],
       [ 3.  ,  4.  ,  5.  , ..., 25.  , 35.  , 40.5 ],
       [ 1.  ,  1.  ,  1.  , ...,  7.  ,  9.  , 13.05],
       ...,
       [ 1.  ,  2.  ,  2.85, ..., 10.  , 11.1 , 12.  ],
       [ 1.  ,  1.  ,  1.  , ...,  5.  ,  5.1 ,  7.  ],
       [ 1.  ,  1.  ,  1.  , ...,  4.  ,  5.  ,  5.1 ]])

In [ ]:
one_demand_prediction = sample_preds[5]
one_demand_prediction


array([1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 4., 4., 5., 5., 6., 7.,
       7., 9.])

In [ ]:
def rarely_run_out_rule(prediction):
    outlier_bound = 3 * np.mean(prediction)
    to_stock = math.ceil(min(prediction[-2], outlier_bound))
    return to_stock


rarely_run_out_rule(one_demand_prediction)

7

In [ ]:
all_stocking_decisions = np.apply_along_axis(rarely_run_out_rule, 1, sample_preds)
(all_stocking_decisions < sample_preds[:, -2]).mean()

0.009091361960745243

In [ ]:
bm5 = log_business_metrics("Quantile regressor", all_stocking_decisions, y_val)
pd.concat([business_metrics, bm1, bm2, bm3, bm4, bm5], axis=0, ignore_index=True)


,Model Name,Understocked Fraction,Understocked Amount,Overstocked Fraction,Overstocked Amount,Utility,MAE,MSE,R2,RMSLE
0,MSE model,0.263425,78859.0,0.644864,101062.0,-337639.0,4.481444,170.217072,0.724259,0.637018
1,MSE model * 1.5,0.128350,36920.0,0.817077,218203.0,-328963.0,6.354563,297.114402,0.518694,0.796495
2,MAE model,0.353069,90968.0,0.480746,60409.0,-333313.0,3.770474,191.123319,0.690392,0.521193
3,MAE model * 1.5,0.170818,42610.0,0.731892,156209.0,-284039.0,4.952152,204.244570,0.669137,0.640890
4,Quantile regressor,0.084313,35904.0,0.878624,395286.0,-502998.0,10.740012,835.487994,-0.353437,1.009825


In [ ]:
for row in [0, 126, 295, 298, 557, 620, 882]:
    print(f"Stocked: {all_stocking_decisions[row]}\nInput: {sample_preds[row, :]}\n")

Stocked: 8
Input: [ 1.  1.  1.  1.  2.  2.  2.  2.  2.  3.  3.  3.  3.  4.  6.  6.  7.  8.
 10.]

Stocked: 3
Input: [1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.45 2.   2.   2.
 3.   3.   3.   3.   4.  ]

Stocked: 141
Input: [  8.95  10.    12.    15.8   18.    29.4   33.3   36.    39.55  42.
  47.25  57.    68.35  72.    78.5   84.2  138.3  141.   187.1 ]

Stocked: 25
Input: [ 1.95  2.    2.    2.    2.75  3.7   4.65  5.    5.55  6.5   8.    8.
  9.   10.   13.   15.   19.   25.   30.  ]

Stocked: 11
Input: [ 1.    2.    2.    2.    2.    3.    3.    3.    4.    4.    4.    4.
  4.35  5.    5.    6.2   8.   10.4  20.  ]

Stocked: 5
Input: [1.   1.   1.   1.   1.75 2.   2.   2.   2.   2.   2.   3.   3.   3.
 4.   4.2  5.   5.   7.05]

Stocked: 6
Input: [1.   1.   1.   2.   2.   2.   2.   2.   2.   2.   2.45 3.   3.   3.
 3.   4.   5.   5.1  7.05]



## References


- [Machine Learning for Business Decision Optimization](https://www.wandb.courses/courses/decision-optimization) - Dan Becker
